# Implementação do Encoder e Decoder Determinístico de Ação


Seguindo o modelo conceitual definido pela arquitetura, em que o World Model opera com pares instantâneos `(St​,at​)`, este notebook irá construir e testar a arquitetura básica de codificação de uma única ação em um instante de tempo `t`. O modelo do enconder traduz o objeto JSON que representa a ação da simulação em um vetor latente compacto `za` e o decoder o reconstrói, a fim de testar a corretude do encoder.

Cada ação tem:
- `time`: timestamp em segundos
- `type`: tipo de ação (mouse_down)
- `object`: objeto afetado (ball)
- `pos`: [x, y] coordenadas da ação

Note que o encoder não codifica o campo `time`, assume-se que a ação e o frame estarão casados temporalmente na etapa do WFM

## 1. Dependências

In [26]:
# Importações necessárias
import torch
import torch.nn as nn
import numpy as np
import json
import random
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Garante a reprodutibilidade
torch.manual_seed(42)

## 2. Definições do Cenário e Codificação

A primeira etapa consiste em codificar uma ação em um tensor do Pytorch. A ausência de ação corresponde à um tensor nulo.

In [20]:
# Dimensões da tela do cenário 1
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600

# Codificação One-Hot
TYPE_ENCODING = {
    'mouse_down': [1.0],  # Ação presente
    'none':  [0.0]        # Ação ausente
}

OBJECT_ENCODING = {
    'ball': [1.0], # Cenário 1 só tem bolas
    'none': [0.0]  # Falta de ação, 0 para que o vetor final da falta de ação seja nulo
}

# Dimensão do vetor de entrada: [type(1)] + [object(1)] + [x_norm(1)] + [y_norm(1)]
INPUT_VECTOR_DIM = 4

LATENT_ACTION_DIM = 16 # Za: Dimensão do Espaço Latente da Ação

In [21]:
# Função de codificação
def encoding_function(action_data: dict = None) -> torch.Tensor:
  """
    Converte um dicionário de ação ou um sinal de 'ausência de ação' em um tensor

    Args:
        action_data: Dicionário de ação (JSON) ou None na ausência dela

    Returns:
        Um tensor PyTorch de dimensão [INPUT_VECTOR_DIM]
  """

  # Caso de ausencia de ação
  if action_data is None:
    type_vector = TYPE_ENCODING['none']
    object_vector = OBJECT_ENCODING['none']
    x_norm = 0.0
    y_norm = 0.0

  # Ação positiva (mouse_down)
  elif action_data:
    type_vector = TYPE_ENCODING['mouse_down']
    object_vector = OBJECT_ENCODING.get(action_data['object'], [0.0]) # Pega o vetor da ball
    pos_x = action_data['pos'][0]
    pos_y = action_data['pos'][1]

    # Normaliza as coordenadas da tela para o intervalo [0, 1]
    x_norm = pos_x / SCREEN_WIDTH
    y_norm = pos_y / SCREEN_HEIGHT

  input_list = type_vector + object_vector + [x_norm, y_norm]

  if len(input_list) != INPUT_VECTOR_DIM:
    raise ValueError(f"Dimensão do vetor inválida: {len(input_list)}")

  return torch.tensor(input_list, dtype=torch.float32)

In [22]:
# Exemplo de ação mouse_down
exemplo_acao = {
    "time": 9.40,
    "type": "mouse_down",
    "object": "ball",
    "pos": [400, 300]
}

# Testando codificação de ação presente
vetor_acao = encoding_function(exemplo_acao)
print("Ação presente (mouse_down):", vetor_acao.tolist())

# Testando codificação de ausência de ação (No-Op)
vetor_none = encoding_function(None)
print("Ausência de ação:", vetor_none.tolist())


Ação presente (mouse_down): [1.0, 1.0, 0.5, 0.5]
Ausência de ação: [0.0, 0.0, 0.0, 0.0]


## 3. Criando o Encoder e Decoder

Agora vamos criar uma rede neural simples que transforma esses 4 features em um embedding mais rico. Depois, vamos fazer o mesmo só que para o movimento contrário. Optou-se por uma rede simples inicialmente, para fins de teste.

In [23]:
class TextEncoder(nn.Module):
  """
  Encoder determinístico (embedding)
  Mapeia o vetor de entrada para o espaço latente de 16 dimensões
  """

  def __init__(self, input_dim=INPUT_VECTOR_DIM, latent_dim=LATENT_ACTION_DIM):
    super().__init__()

    self.latent_dim = latent_dim

    self.net = nn.Sequential(
      nn.Linear(input_dim, 16),
      nn.ReLU(),
      nn.Linear(16, latent_dim)
    )

  def forward(self, x):
    return self.net(x)

In [24]:
class TextDecoder(nn.Module):
  """
  Decoder determinístico (reconstrução)
  Mapeia o vetor latente de volta para o espaço da ação original
  """

  def __init__(self, latent_dim=LATENT_ACTION_DIM, output_dim=INPUT_VECTOR_DIM):
    super().__init__()

    self.net = nn.Sequential(
      nn.Linear(latent_dim, 16),
      nn.ReLU(),
      nn.Linear(16, output_dim)
    )

  def forward(self, z):
    return self.net(z)

## 4. Geração de um dataset sintético simples

Como a estrutura das ações é muito simples, é mais vantajoso montar um dataset sintético com mais ações do que usar o dataset real com 200 simulações. Note que na simulação real, menos de 1% dos frames terão uma ação correspondente, logo a proporção de ações positivas (mouse_click) em relação a quantidade de frames é de 1%, a fim de espelhar o comportamento real da simulação.

In [32]:
NUM_SAMPLES = 2000     # Número de amostras sintéticas
BATCH_SIZE = 64        # Tamanho do lote

# Lista para armazenar os vetores de input
synthetic_inputs = []

# Gerar amostras de clique
for _ in range(20): # 1% de 2000

  pos_x = random.randint(50, 750)
  pos_y = random.randint(50, 500)

  # Monta a estrutura de dados
  action_data = {
    "type": "mouse_down",
    "object": "ball",
    "pos": [pos_x, pos_y]
  }

  # Codifica e armazena
  synthetic_inputs.append(encoding_function(action_data))


# Gerar amostras de Ausência de Ação
for _ in range(1980): # 99% de 2000
  # Quando não tem ação, passa um None à função de encoding
  synthetic_inputs.append(encoding_function(None))

In [35]:
# Teste de entradas da lista de ações sintéticas
print(synthetic_inputs[0])
print(synthetic_inputs[50])

tensor([1.0000, 1.0000, 0.2175, 0.7183])
tensor([0., 0., 0., 0.])


## 5.